<a href="https://colab.research.google.com/github/dagaravitoj/aud-crawler/blob/master/Creaci%C3%B3n_documentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# log
EL objetivo de este script es alimentar las bases de contenido de EducALL. Para iniciar con la sugerencia de contenido.

[x] Listas de reproducción a enlaces

[x] Extracción de metadatos de Youtube para recomendación no personalizada

[] Key Frames

[x] Descargar subtitulos

[] Encuesta de Intereses

[x] Reescribir la importación del audio

[] Jerrquia de carpetas para exportación

[] Probar otra forma de extraer los screens
ffmpeg -ss 4 -i input.avi -s 320x240 -frames:v 1 output.jpg

http://trac.ffmpeg.org/wiki/Create%20a%20thumbnail%20image%20every%20X%20seconds%20of%20the%20video


old 
https://colab.research.google.com/drive/1CHno4kow8wYVccyrhUYQHeuLB1c9iBF1?authuser=1qº

In [ ]:
!pip install pydub --quiet
!pip install gtts --quiet
!pip install playsound --quiet
# !pip install youtube-mp3-downloader
#!pip install youtube_dl
#!pip install ffmpeg
#!pip install katna
!pip install python-docx --quiet
#!pip install youtube-transcript-api
!pip install imagehash --quiet
!pip install yt-dlp --quiet
#!pip install docx2pdf


     |████████████████████████████████| 5.6 MB 7.5 MB/s 
     |████████████████████████████████| 812 kB 7.9 MB/s 
     |████████████████████████████████| 2.6 MB 8.1 MB/s 
     |████████████████████████████████| 112 kB 55.9 MB/s 
     |████████████████████████████████| 357 kB 58.9 MB/s 
     |████████████████████████████████| 218 kB 43.0 MB/s 
     |████████████████████████████████| 2.0 MB 41.2 MB/s 


# Conectarse a Google Drive

In [ ]:
#import os
#import pandas as pd
#import subprocess
#from google.colab import auth
#auth.authenticate_user()
#import gspread
#from oauth2client.client import GoogleCredentials
#gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# Playlist a descargar


https://www.youtube.com/c/yotelocuento/playlists


https://www.youtube.com/watch?v=9hoeuYzYIGo&list=PL6UDGdAJCq3xyPK3urik0Lp4PCQcpZOYi

#Archivo con las listas a descargar, pendiente automatizar
https://docs.google.com/spreadsheets/d/1OBLavC7l3pmgeCcEn83RJMqf_2nfV5CY6Li0-GTz4_A/edit

In [ ]:
import pandas as pd
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1OBLavC7l3pmgeCcEn83RJMqf_2nfV5CY6Li0-GTz4_A/edit#gid=1236566438')
#wb = gc.open_by_key('1CkoGczcFuESjPijsKA4_5aKBimOFbqL-')
sheet = wb.worksheet('Crawler 2022')
data = sheet.get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]
df.head()
df_clean=df[pd.to_numeric(df.docs)==0]
df_clean.reset_index(inplace=True)

playlist_id=df_clean.iloc[0,1]
playlist_name=df_clean.iloc[0,2]
canal_name=df_clean.iloc[0,3]

playlist_id

canal_name
playlist_name

'Todo Khan'

## Directorios de trabajo

In [ ]:
import os
import re
#path="/content/drive/My Drive/2020/Hackathon EducAll/Tech/Audio/EducALL Contenido CC/Procesados/"
import pandas as pd
pd.set_option('display.max_colwidth', None)

path="/gdrive/MyDrive/ContenidosEducALL/"
path_share ='/gdrive/Shareddrives/ContenidosEducALL/'

out_path=os.path.join(path,playlist_name)
result_path=os.path.join(path_share,'0.Salida',playlist_name)
os.mkdir(result_path)


## Cargue de archivo con metadatos 

In [ ]:
xlsx=[]
for file in os.listdir(result_path):
    if file.endswith(".xlsx"):
      xlsx.append(os.path.join(result_path, file))

video_data=pd.read_excel(xlsx[0], index_col=0) 

# Cargue de archivo total

In [ ]:
# Archivos de archivos con key_words (de spacy)
key_words_files=[]

for file in os.listdir(path):
    if file.startswith("key"):
      key_words_files.append(file)

key_words_files


li=[]
for filename in key_words_files:
    df = pd.read_excel(os.path.join(path,filename), index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True) 

frame


In [ ]:
#frame.columns
frame.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1','Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1'],axis=1,inplace=True)



In [ ]:
len(frame) #Ejecución: No Requerida
#frame.to_excel(os.path.join(out_path,playlist_name+".xlsx"))

('/gdrive/MyDrive/ContenidosEducALL/Todo Khan', 'Todo Khan')

In [ ]:
#df_subtitles=pd.read_excel(os.path.join(path,'df_subtitles.xlsx'))

#df_subtitles=pd.read_excel(os.path.join(path,'key_words 0 600.xlsx'))
video_data=frame

video_data.columns
video_data['filename'].value_counts()

Algebra I.xlsx                                               460
Pre-álgebra.xlsx                                            305
Economía: Mercados Financieros y de capital.xlsx            278
Algebra II.xlsx                                              248
Geometría.xlsx                                              214
Cálculo Integral.xlsx                                       211
Química Orgánica.xlsx                                      200
Matemáticas básicas F Carlos Slim.xlsx                     156
Cálculo Diferencial.xlsx                                    147
Pre-cálculo.xlsx                                            136
Àlgebra Lineal.xlsx                                         132
Probabilidad y estadística.xlsx                             118
Fisíca.xlsx                                                 118
Matemáticas Elementales.xlsx                                111
Principios de álgebra.xlsx                                  111
Cálculo multivariable.xl

# 📃Función para crear documentos

In [ ]:
rate='0.25'
video_data['filename'].replace(".xlsx","",regex=True, inplace = True)
videos_carpetas=video_data['filename'].drop_duplicates()
video_data_clean=video_data.drop_duplicates(subset='video_id',keep='last')
iii=videos_carpetas.iloc[2]
video_data_sub=video_data[video_data['filename']==iii]

In [ ]:
import imagehash
from PIL import Image

def calculate_hash(dir):
    """Generate Hash Values for all images in a directory

    Args:
        dir (str): Directory to search for images

    Returns:
        hash_values (list): List of hash values for all images in the directory
        image_ids (list): List of image ids for all images in the directory
    """

    hash_values = []
    image_ids = []

    for file in pngs:

        #path = os.path.join(dir, file)
        img = Image.open(file)

        #hash = imagehash.average_hash(img)
        hash = imagehash.phash(img)
        #hash = imagehash.whash(img) #muy sensible
        #hash = imagehash.dhash(img)
        #hash = imagehash.colorhash(img)

        hash_values.append(hash)
        image_ids.append(file)

    return image_ids, hash_values

def prepare_dataframe(image_ids, hash_values):

    # Create DataFrame with hash values and image ids
    df = pd.DataFrame(
        {
            "image_ids": image_ids, 
            "hash_values": hash_values, 
            }
    )

    # Create new columns in df with image_ids having hash difference value=0 
    for i in range(len(df.image_ids)):
        df[f"diff_{image_ids[i]}"] = 0

    return df

def calculate_differences(df):

    # Obtain difference for every image_id one by one
    for i in range(len(df.hash_values)):
        differences = []

        for j in range(len(df.hash_values)):
            differences.append(df.hash_values[i] - df.hash_values[j])

        # Store the difference values for every image_id
        df.iloc[i, 2:] = differences

    return df

# Función principal

# Función principal parchada

In [ ]:
# iii es el nombre de la lista
#def crea_docs(iii):

from docx import Document
from docx.shared import Inches
import time
import re
#from docx2pdf import convert
## Creación del documento

#for i in range(len(videos_carpetas)):
#doc_num=0
#i=20
for i in range(25,28): # x, y(-1)
#pngs=[]
#def crea_docs(i, doc_num):
  iii=videos_carpetas.iloc[i]
  video_data_sub=video_data[video_data['filename']==iii]
  print('número ' + str(i) + ' carpeta ' + str(iii))
  # videos que tienen subtitulos
  video_data_sub_str=subs=video_data_sub[[len(str(i)) >  10 for i in video_data_sub['str']]]
  #len(video_data_sub)
  #len(video_data_sub_str)


  video_data_docx_filter=video_data_sub_str
  video_data_docx_filter.drop_duplicates(subset ="video_id",inplace=True)
  #video_data_docx_filter.columns
  for i in (range(doc_num,len(video_data_docx_filter.index))):
    video_id=video_data_docx_filter.video_id.iloc[i]
    video_name=video_data_docx_filter.Title.iloc[i]
    canal_name=str(video_data_docx_filter.chanel.iloc[i])
    #print(video_id)
    #print(video_name)
    
    pngs=[]
    
    for file in os.listdir(os.path.join(path,iii)):
        if file.endswith(".png") and (video_id in file):
          pngs.append(os.path.join(path,iii, file))

    pngs.sort()
    #print(pngs)
    # Obtain image_ids and respective hash values
    image_ids, hash_values = calculate_hash(pngs)
    # Obtain dataframe
    df = prepare_dataframe(image_ids, hash_values)
    #df.head(2)
    df = calculate_differences(df)
    df.reset_index(inplace=True)
    df.sort_values(by=['image_ids'])
    df = df.drop_duplicates('hash_values', keep='first')
    pngs_select=[]
    pngs_select=list(df.image_ids[df.hash_values.diff(periods=1)>1])
    #print(pngs_select)
  #texto de subtitulos para el documento
    S=video_data_docx_filter['str'][video_data_docx_filter['video_id']==video_id]
    T=video_data_docx_filter['start'][video_data_docx_filter['video_id']==video_id]
    C=video_data_docx_filter['key_words'][video_data_docx_filter['video_id']==video_id]

    #Segundo de la imagen
    frames=list(map(int, ([x[-7:-4] for x in pngs_select][:-1] )))
    #frames=[0] + frames
    seg=[x*1/float(rate) for x in frames]
    tiempo=[time.strftime("%H:%M:%S",time.gmtime(int(x))) for x in seg]+['FIN']
    #Texto en cada imagen
    # falla porque esta almacenando doble los subtitulos (no sé porqué)
    try:
      start=[float(x) for x in [re.sub("[^0-9|\.]", "", x) for x in str(T).split('///')]]
      text=[re.sub("\nName: str, dtype: object", "", x) for x in str(S).split('///')]
    except:
      value=[x.count('.')==2 for x in str(T).split('///') ]
      start=[(x) for x in [re.sub("[^0-9|\.]", "", x) for x in str(T).split('///')]]
      text=[re.sub("\nName: str, dtype: object", "", x) for x in str(S).split('///')]
      dos_puntos = pd.DataFrame(list(zip(start, text)),columns =['start', 'text'])
      dos_puntos.drop(range(dos_puntos.index[value].tolist()[0]+1, len(dos_puntos)), axis=0,inplace=True)
      dos_puntos.iloc[len(dos_puntos)-1,:][0]=dos_puntos.iloc[len(dos_puntos)-1,:][0][:-4]
      start=dos_puntos['start'].tolist()
      text=dos_puntos['text'].tolist()


    strdf=pd.DataFrame(pd.cut(start, bins=[0]+seg),text).reset_index()
    strdf.columns=['srt','start']

    strdf_gr=strdf.groupby(['start']).agg({'srt': ' '.join})
    strdf_gr.reset_index(inplace=True)

    str_doc=pd.concat([strdf_gr.srt,pd.Series(['Fin'])])

    str_doc=pd.DataFrame(str_doc).reset_index()
    str_doc.columns=['index','str']

    document = Document()
    document.add_picture('/gdrive/MyDrive/2020/Hackathon EducALL/Branding/Logo/Educall logo slogan 1.png', width=Inches(2.5))
    document.add_heading(iii, 0)
    document.add_heading(video_name, 1)
    document.add_paragraph('Obtenido del canal de Youtube '+canal_name + ' distribuido bajo licencias Creative Commons')
    document.add_heading('Palabras Clave', 2)
    document.add_paragraph(re.sub("\\nName: key_words, dtype: object", "",str(C)))


    #document.add_paragraph(str(S).split('///'))
    document.add_heading('Imágenes de apoyo', 2)


  #  print(len(pngs_select))
  #  print(len(str_doc.str))

    for j in range(0, len(pngs_select)):
      document.add_heading(tiempo[j],3)
      document.add_picture(pngs_select[j], width=Inches(4.5))
      document.add_paragraph(str_doc.str[j])
    document.save(os.path.join(path,'0.Salida',iii,iii +' '+video_name+' '+video_id+'.docx'))
    #convert(os.path.join(path,'0.Salida',iii,iii +' '+video_name+' '+video_id+'.docx'))
    print(str(i) + ' de ' + str(len(video_data_docx_filter)) +' '+ video_id +' ' +video_name)
    doc_num=i+1


número 25 carpeta Microeconomía
número 26 carpeta Principios de álgebra
número 27 carpeta Algebra I


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newfra

163 de 459 yrG8hQV-ZX8 Similitud entre pendiente y triángulo 4
164 de 459 L_tyTHNexhU Triángulos similares para probar pendiente constante
165 de 459 XlZ-JDkun5U Encontrando la intersección en una función lineal con una tabla
166 de 459 JSlVrJKbXBU Encontrando la intersección X de una línea
167 de 459 2y2p9gFKAX0 Graficando usando Intersecciones
168 de 459 4oKA_Uja9jU Gráficas utilizando intersecciones en X e Y
169 de 459 J67akqk4CoY Interpretando intersecciones de funciones lineales
170 de 459 HEF8Eepm5C8 Intersección X,Y
171 de 459 vz6VLzNRIGU Intersección X,Y 2
172 de 459 xoo0KRCLKFk Expresiones algebráicas con división de fracciones
173 de 459 eQV3VOgaKYk Expresiones algebráicas con suma de fracciones
174 de 459 WM3lZWweq44 Ejercicios de variables dependientes e independientes. Ejemplo 1
175 de 459 kBkCBk5sEvc Ejercicios de variables dependientes e independientes. Ejemplo 2
176 de 459 9uiCCGYEVY4 Ejercicios de variables dependientes e independientes. Ejemplo 3
177 de 459 NyMNzn10Re

In [ ]:
#path_docx=os.path.join(path,'0.Salida',iii,iii) #Ejecución: No requerida
#convert(path_docx)
crea_docs(2,0)
#pasar a 2


# Celdas de respaldo


# 📃Creación de Documento
https://python-docx.readthedocs.io/en/latest/
pip install python-docx


In [ ]:
from docx import Document
from docx.shared import Inches
import time
import re

#video_data_sub.head()
# documentos pendientes
file_docxs=[]
docxs=[]
for file in os.listdir(os.path.join(path,'0.Salida',iii)):
    if file.endswith(".docx"):
      file_docxs.append(file)
      docxs.append(file[-16:-5])

#Borrado de archivos, para reescribir
#for docx in file_docxs:
#  os.remove(os.path.join(path,'0.Salida',iii,docx))

pendientes = list(set(video_data_sub.video_id) - set(docxs))
pendientes=pendientes[1:]

video_data_docx_filter=video_data_sub[video_data_sub['video_id'].isin(pendientes)]
#video_data_docx_filter.video_id[video_data_docx_filter.video_id=='9Y4FFK7b-M']

for i in (video_data_docx_filter.index):
  video_id=video_data_sub.video_id[i]
  video_name=video_data_sub.Title[i]
  
  pngs=[]
  for file in os.listdir(os.path.join(path,iii)):
      if file.endswith(".png") and (video_id in file):
        pngs.append(os.path.join(path,iii, file))
  pngs.sort()
  # Obtain image_ids and respective hash values
  image_ids, hash_values = calculate_hash(pngs)
  # Obtain dataframe
  df = prepare_dataframe(image_ids, hash_values)

  df = calculate_differences(df)
  df.reset_index(inplace=True)
  df.sort_values(by=['image_ids'])
  df = df.drop_duplicates('hash_values', keep='first')

  pngs_select=list(df.image_ids[df.hash_values.diff(periods=1)>1])
#texto de subtitulos para el documento
  S=video_data_sub['str'][video_data_sub['video_id']==video_id]
  T=video_data_sub['start'][video_data_sub['video_id']==video_id]
  C=video_data_sub['key_words'][video_data_sub['video_id']==video_id]

  #Segundo de la imagen
  frames=list(map(int, ([x[-7:-4] for x in pngs_select][:-1] )))
  #frames=[0] + frames
  seg=[x*1/float(rate) for x in frames]
  tiempo=[time.strftime("%H:%M:%S",time.gmtime(int(x))) for x in seg]+['FIN']
  #Texto en cada imagen
  start=[float(x) for x in [re.sub("[^0-9|\.]", "", x) for x in str(T).split('///')]]
  text=[re.sub("\nName: str, dtype: object", "", x) for x in str(S).split('///')]

  strdf=pd.DataFrame(pd.cut(start, bins=[0]+seg),text).reset_index()
  strdf.columns=['srt','start']

  strdf_gr=strdf.groupby(['start']).agg({'srt': ' '.join})
  strdf_gr.reset_index(inplace=True)

  str_doc=pd.concat([strdf_gr.srt,pd.Series(['Fin'])])

  str_doc=pd.DataFrame(str_doc).reset_index()
  str_doc.columns=['index','str']

  document = Document()
  document.add_picture('/gdrive/MyDrive/2020/Hackathon EducALL/Branding/Logo/Educall logo slogan 1.png', width=Inches(2.5))
  document.add_heading(iii, 0)
  document.add_heading(video_name, 1)
  document.add_paragraph('Obtenido del canal de Youtube '+canal_name + ' distribuido bajo licencias Creative Commons')
  document.add_heading('Palabras Clave', 2)
  document.add_paragraph(re.sub("\\nName: key_words, dtype: object", "",str(C)))


  #document.add_paragraph(str(S).split('///'))
  document.add_heading('Imágenes de apoyo', 2)


#  print(len(pngs_select))
#  print(len(str_doc.str))

  for j in range(0, len(pngs_select)):
    document.add_heading(tiempo[j],3)
    document.add_picture(pngs_select[j], width=Inches(4.5))
    document.add_paragraph(str_doc.str[j])
  document.save(os.path.join(path,'0.Salida',iii,iii +' '+video_name+' '+video_id+'.docx'))
  print(str(i) + ' de ' + str(len(video_data_docx_filter)) +' '+ video_id)


NameError: ignored

# ⚙ en progreso
## concatenar arichvos de excel para recomendación 

busqueda recursiva en sub carpetas
https://stackoverflow.com/questions/5817209/browse-files-and-subfolders-in-python

In [ ]:
xlsx=[]
for file in os.listdir(result_path):
    if file.endswith(".xlsx"):
      xlsx.append(os.path.join(result_path, file))

xlsx

['/gdrive/MyDrive/ContenidosEducALL/0.Salida/Guías para Padres/Guías para Padres.xlsx']

# Sopa de letras
https://scipython.com/blog/making-a-word-search-puzzle-in-python/

In [ ]:
video_data['filename'].replace(".xlsx","",regex=True, inplace = True)
videos_carpetas=video_data['filename'].drop_duplicates()
video_data_clean=video_data.drop_duplicates(subset='video_id',keep='last')
iii=videos_carpetas.iloc[2]
video_data_sub=video_data[video_data['filename']==iii]
